# ftp.census.gov Coverage

This notebook uses a listing of files from `ftp2.census.gov` to determine how many of them were archived by the Wayback Machine at their web "frontend" `www2.census.gov`. The list was created by Andrew Berger using lftp to copy files from ftp2.census.gov.

## Get a DataFrame

Create a DataFrame from the ISO-8859-1 encoded list of files.

In [24]:
import gzip
import pandas

urls = [url.strip().decode("iso-8859-1") for url in gzip.open('data/all-downloaded-census-2025-03-29.txt.gz')]
df = pandas.DataFrame({"ftp_url": urls})
df

,ftp_url
0,ftp2.census.gov/econ/esp/2012/esp2012_table7.xlsx
1,ftp2.census.gov/econ/esp/2012/Table 4.xlsx
2,ftp2.census.gov/econ/esp/2012/esp2012_table6.xlsx
3,ftp2.census.gov/econ/esp/2012/Table 6.xlsx
4,ftp2.census.gov/econ/esp/2012/esp2012_table8.xlsx
...,...
4496866,ftp2.census.gov/acs2011_1yr/summaryfile/Sequen...
4496867,ftp2.census.gov/acs2011_1yr/summaryfile/ACS201...
4496868,ftp2.census.gov/acs2011_1yr/summaryfile/ACS_20...
4496869,ftp2.census.gov/acs2011_1yr/summaryfile/Sequen...


Use the FTP URL to create a Web URL:

In [36]:
import re

df['web_url'] = df.ftp_url.apply(lambda s: re.sub('^ftp2', 'https://www2', s))
df

,ftp_url,web_url
0,ftp2.census.gov/econ/esp/2012/esp2012_table7.xlsx,https://www2.census.gov/econ/esp/2012/esp2012_...
1,ftp2.census.gov/econ/esp/2012/Table 4.xlsx,https://www2.census.gov/econ/esp/2012/Table 4....
2,ftp2.census.gov/econ/esp/2012/esp2012_table6.xlsx,https://www2.census.gov/econ/esp/2012/esp2012_...
3,ftp2.census.gov/econ/esp/2012/Table 6.xlsx,https://www2.census.gov/econ/esp/2012/Table 6....
4,ftp2.census.gov/econ/esp/2012/esp2012_table8.xlsx,https://www2.census.gov/econ/esp/2012/esp2012_...
...,...,...
4496866,ftp2.census.gov/acs2011_1yr/summaryfile/Sequen...,https://www2.census.gov/acs2011_1yr/summaryfil...
4496867,ftp2.census.gov/acs2011_1yr/summaryfile/ACS201...,https://www2.census.gov/acs2011_1yr/summaryfil...
4496868,ftp2.census.gov/acs2011_1yr/summaryfile/ACS_20...,https://www2.census.gov/acs2011_1yr/summaryfil...
4496869,ftp2.census.gov/acs2011_1yr/summaryfile/Sequen...,https://www2.census.gov/acs2011_1yr/summaryfil...


## Lookup in Wayback

To look up a URL in the Wayback Machine we can use the [wayback](https://wayback.readthedocs.io/en/stable/index.html) module which talks to the Wayback Machine's [CDX API](https://archive.org/developers/wayback-cdx-server.html).

In [37]:
import wayback

wb = wayback.WaybackClient()

In [38]:
df.web_url[0]

'https://www2.census.gov/econ/esp/2012/esp2012_table7.xlsx'

In [40]:
for result in wb.search('https://www2.census.gov/econ/esp/2012/esp2012_table7.xlsx'):
    print(result)

CdxRecord(key='gov,census)/econ/esp/2012/esp2012_table7.xlsx', timestamp=datetime.datetime(2017, 7, 22, 6, 4, 38, tzinfo=datetime.timezone.utc), url='http://www2.census.gov/econ/esp/2012/esp2012_table7.xlsx', mime_type='unk', status_code=302, digest='3I42H3S6NNFQ2MSVX7XZKYAYSCX5QBYJ', length=418, raw_url='https://web.archive.org/web/20170722060438id_/http://www2.census.gov/econ/esp/2012/esp2012_table7.xlsx', view_url='https://web.archive.org/web/20170722060438/http://www2.census.gov/econ/esp/2012/esp2012_table7.xlsx')
CdxRecord(key='gov,census)/econ/esp/2012/esp2012_table7.xlsx', timestamp=datetime.datetime(2017, 8, 17, 11, 48, 24, tzinfo=datetime.timezone.utc), url='https://www2.census.gov/econ/esp/2012/esp2012_table7.xlsx', mime_type='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet', status_code=200, digest='6Y6373IAFSDMMIOTXDXHGACICLMA2Y4J', length=30557, raw_url='https://web.archive.org/web/20170817114824id_/https://www2.census.gov/econ/esp/2012/esp2012_table7.xls

This shows that there were seven snapshots taken of this URL. However, it's important to look at the `status_code` for each snapshot. The first one is a `302` redirect to a page that was not archived.

* https://web.archive.org/web/20170722060438id_/http://www2.census.gov/econ/esp/2012/esp2012_table7.xlsx

The 5th and 6th snapshots are a `403` access denied error.

* https://web.archive.org/web/20241215202549id_/https://www2.census.gov/econ/esp/2012/esp2012_table7.xlsx

So we need a function that will return the timestamps for all 200 OK responses, or an empty list in the case of the URL not being archived in the Wayback Machine.

In [65]:
def snapshots(url):
    return [result.timestamp for result in wb.search(url) if result.status_code == 200]

In [66]:
snapshots('https://www2.census.gov/econ/esp/2012/esp2012_table7.xlsx')

[datetime.datetime(2017, 8, 17, 11, 48, 24, tzinfo=datetime.timezone.utc),
 datetime.datetime(2021, 3, 26, 18, 7, 8, tzinfo=datetime.timezone.utc),
 datetime.datetime(2025, 3, 24, 6, 42, 7, tzinfo=datetime.timezone.utc)]

In [67]:
snapshots('https://www2.census.gov/econ/esp/2012/esp2012_table7-bogus.xlsx')

[]

## Sample the Data

There are 4,496,871 URLs. If we checked one per second, one at a time, it would take 52 days. In this notebook we're just going to sample them to get a sense of the coverage. 

According to [this](https://www.calculator.net/sample-size-calculator.html?type=1&cl=95&ci=5&pp=50&ps=4496871&x=Calculate) calculator, if we want 95% confidence with 5% margin of error we can randomly sample 385 URLs. This should be good enough to get a sense of the coverage to start.

In [68]:
sample = df.sample(385)
sample

,ftp_url,web_url
3483319,ftp2.census.gov/programs-surveys/acs/summary_f...,https://www2.census.gov/programs-surveys/acs/s...
2086146,ftp2.census.gov/geo/maps/blk2000/st48_Texas/Co...,https://www2.census.gov/geo/maps/blk2000/st48_...
4325700,ftp2.census.gov/acs2010_3yr/summaryfile/2008-2...,https://www2.census.gov/acs2010_3yr/summaryfil...
1918233,ftp2.census.gov/geo/maps/blk2000/st01_Alabama/...,https://www2.census.gov/geo/maps/blk2000/st01_...
447922,ftp2.census.gov/geo/tiger/TIGER2020PL/LAYER/FA...,https://www2.census.gov/geo/tiger/TIGER2020PL/...
...,...,...
2010740,ftp2.census.gov/geo/maps/blk2000/st29_Missouri...,https://www2.census.gov/geo/maps/blk2000/st29_...
4408869,ftp2.census.gov/EEO_2014_2018/EEO_Tables_By_Al...,https://www2.census.gov/EEO_2014_2018/EEO_Tabl...
1763263,ftp2.census.gov/geo/maps/dc10map/GUBlock/st48_...,https://www2.census.gov/geo/maps/dc10map/GUBlo...
4389598,ftp2.census.gov/EEO_2014_2018/EEO_Tables_By_Al...,https://www2.census.gov/EEO_2014_2018/EEO_Tabl...


And now we can apply our function to it to create a new column holding the archive timestamps if available:

In [ ]:
df['archived'] = df.web_url.apply(snapshots)